In [ ]:
!pip install colabcode
!pip install fastapi

In [ ]:
from PIL import Image
import tensorflow
from google.colab import drive
import numpy as np
import os
import cv2

import tensorflow as tf
from tensorflow import keras
from glob import glob

from colabcode import ColabCode

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import pickle

app = FastAPI()

origins = [
    "http://localhost:3000",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get('/')
def index():
    return {'message': 'This is the homepage of the API '}

drive.mount("/content/drive/")

data=[] 
labels=[]
labelsMap=[]

count = 0

dataset_url = glob(f"/content/drive/My Drive/DL/*/", recursive=True)

for dir in dataset_url:
  # print(dir)
  labelsMap.append(dataset_url[count][27:].replace('s/', ''))
  temp = os.listdir(dir)
  for animal in temp:
    imag=cv2.imread(dir + animal)
    img_from_ar = Image.fromarray(imag, 'RGB')     
    resized_image = img_from_ar.resize((50, 50)) 
    data.append(np.array(resized_image))     
    labels.append(count)
  count = count + 1

# print(data)
# print(labels)
# print(labelsMap)

animals=np.array(data) 
labels=np.array(labels)

np.save("animals", animals) 
np.save("labels", labels)

s=np.arange(animals.shape[0]) 
np.random.shuffle(s) 
animals=animals[s] 
labels=labels[s] 

num_classes=len(np.unique(labels))
data_length=len(animals)

(x_train,x_test)=animals[(int)(0.1*data_length):],animals[:(int)(0.1*data_length)]
x_train = x_train.astype('float32')/255 
x_test = x_test.astype('float32')/255 
train_length=len(x_train) 
test_length=len(x_test)

(y_train, y_test)=labels[(int)(0.1*data_length):],labels[:(int)(0.1*data_length)] 

y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)

print("End of dataset prepration!!!")

from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

model=Sequential() 
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50, 50, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2)) 
model.add(Dropout(0.2)) 
model.add(Flatten()) 
model.add(Dense(500,activation="relu")) 
model.add(Dropout(0.2)) 
model.add(Dense(6, activation="softmax")) 
model.summary()

# compile the model 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=50, epochs=100,verbose=1)

score = model.evaluate(x_test, y_test, verbose=1) 
print('\n', 'Test accuracy:', score[1]) 

def convert_to_array(img):     
  im = cv2.imread(img)     
  img = Image.fromarray(im, 'RGB')     
  image = img.resize((50, 50))     
  return np.array(image) 

def get_animal_name(label):      
  for x in range(len(labelsMap)):
    if label==x: 
      return labelsMap[x]     

@app.get('/predict')
def predict_animal(file):     
    print("Predicting .................................")     
    ar=convert_to_array(file)     
    ar=ar/255     
    label=1     
    a=[]     
    a.append(ar) 
    a=np.array(a)     
    score=model.predict(a,verbose=1)     
    print(score)     
    label_index=np.argmax(score)     
    print(label_index)     
    acc=np.max(score)
    animal=get_animal_name(label_index)     
    print(animal)     
    print("The predicted Animal is a "+ animal +" with accuracy = " + str(acc))
    return {'animal_category': animal}

predict_animal("/content/drive/My Drive/DL/rabbits/rabbit4.jpeg") 
predict_animal("/content/drive/My Drive/DL/dogs/dog4.jpeg") 
predict_animal("/content/drive/My Drive/DL/cats/cat4.jpeg") 

server = ColabCode(port=10000, code=False)
server.run_app(app=app)
